https://www.kaggle.com/code/mcarujo/synthetic-data-generation-sdv-tutotial

In [ ]:
!pip install sdv==0.13.1 -q


In [ ]:
import warnings

import pandas as pd

from sdv.evaluation import evaluate
from sdv.metrics.tabular import CSTest, KSTest, LogisticDetection, SVCDetection
from sdv.tabular import CTGAN, TVAE, CopulaGAN, GaussianCopula

warnings.filterwarnings("ignore")


In [ ]:
!pip install -U https://github.com/pandas-profiling/pandas-profiling/archive/master.zip

!pip install -U pandas-profiling

import pandas_profiling as pp
from pandas_profiling import ProfileReport

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Using cached https://github.com/pandas-profiling/pandas-profiling/archive/master.zip
  Preparing metadata (setup.py) ... done
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pandas_profiling import ProfileReport


In [ ]:
profile = ProfileReport(
    real_data, title="Profiling Real Adult Data", html={"style": {"full_width": True}}
)
profile.to_file("real_data_report_ctgan.html")
profile.to_notebook_iframe()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
!pip show sdv


Name: sdv
Version: 0.13.1
Summary: Synthetic Data Generation for tabular, relational and time series data.
Home-page: https://github.com/sdv-dev/SDV
Author: MIT Data To AI Lab
Author-email: dailabmit@gmail.com
License: MIT license
Location: /usr/local/lib/python3.8/dist-packages
Requires: copulas, ctgan, deepecho, Faker, graphviz, numpy, pandas, rdt, sdmetrics, tqdm
Required-by: 


In [ ]:
import numpy as np
real_data = pd.read_csv("/content/PS1D_train.csv")
real_data.drop(['Index'], axis=1, inplace=True)
print(real_data.shape)
real_data.head()


(835, 9)


,C1,C2,C3,C4,C5,C6,C7,C8,Target
0,1,0.665,0.500,0.175,1.2975,0.6075,0.3140,0.3150,9
1,2,0.535,0.460,0.145,0.7875,0.3395,0.2005,0.2000,8
2,1,0.625,0.495,0.155,1.0485,0.4870,0.2120,0.3215,11
3,2,0.625,0.490,0.110,1.1360,0.5265,0.1915,0.2925,9
4,3,0.410,0.300,0.090,0.2800,0.1410,0.0575,0.0750,8


In [ ]:


# y=data.iloc[:, -1]
# # X
# scalar.fit(num)
# scaled_data = scalar.transform(num)

# # Importing PCA
# from sklearn.decomposition import PCA

# pca = PCA(n_components = 0.95)
# pca.fit(scaled_data)
# x_pca = pca.transform(scaled_data)

# x_pca.shape


In [ ]:
model_tvae=TVAE()

In [ ]:
model_tvae.fit(real_data)

In [ ]:
real_sub_data = pd.read_csv("/content/PS1D_sample_submission.csv")

In [ ]:
new_data_tvae = model_tvae.sample(10000)

In [ ]:
evaluate(
    new_data_tvae,
    real_data,
    metrics=['KSTest'],
    aggregate=False,
)

,metric,name,raw_score,normalized_score,min_value,max_value,goal,error
0,KSTest,Inverted Kolmogorov-Smirnov D statistic,0.930041,0.930041,0.0,1.0,MAXIMIZE,None


In [ ]:
X=new_data_tvae.iloc[:,:-1]
y=new_data_tvae.iloc[:,-1]
X

,C1,C2,C3,C4,C5,C6,C7,C8
0,3,0.383,0.268,0.135,0.3171,0.1357,0.0175,0.0586
1,3,0.428,0.311,0.098,0.3835,0.0882,0.0505,0.0572
2,2,0.643,0.420,0.192,0.6122,0.4369,0.2661,0.5617
3,3,0.268,0.226,0.114,0.6350,0.0050,0.0565,0.0210
4,1,0.582,0.455,0.149,1.0124,0.3655,0.1670,0.3698
...,...,...,...,...,...,...,...,...
9995,1,0.586,0.518,0.157,2.0525,0.6334,0.2148,0.3719
9996,1,0.413,0.389,0.124,0.5443,0.3133,0.1074,0.2371
9997,3,0.411,0.226,0.173,0.0995,0.1300,0.0847,0.1191
9998,2,0.706,0.504,0.174,1.0613,0.6552,0.3667,0.4026


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)

In [ ]:
# Importing standardscalar module
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

scalar = StandardScaler()
enc=OneHotEncoder()

# fitting
cat_feats=['C1']
num_feats = ['C2','C3','C4','C5','C6','C7','C8']

from sklearn.pipeline import Pipeline
num_trans=Pipeline([('std_scaler', StandardScaler())])
cat_trans= OneHotEncoder(handle_unknown='ignore') 

from sklearn.compose import ColumnTransformer
tot_trans=ColumnTransformer(transformers=[('num',num_trans,num_feats),('cat',cat_trans,cat_feats)],remainder='passthrough')
df_trainp=tot_trans.fit_transform(X_train)
df_testp=tot_trans.transform(X_test)

In [ ]:
df1=pd.DataFrame(df_trainp)
dft=pd.DataFrame(df_testp)

In [ ]:
df1

,0,1,2,3,4,5,6,7,8,9
0,0.129991,0.517896,0.377614,0.982364,-0.726874,-0.083146,-0.448406,1.0,0.0,0.0
1,1.478680,1.080980,0.974713,1.302151,0.484395,0.747994,1.328231,0.0,1.0,0.0
2,-0.017139,1.262312,-1.103191,1.900323,0.519621,0.324710,-0.155730,0.0,1.0,0.0
3,0.048252,-0.197890,0.282078,-0.151283,-1.509897,-1.240186,-0.772332,0.0,0.0,1.0
4,0.203556,0.403370,-0.338904,0.752283,-0.062110,-0.229705,1.283263,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
7995,-0.891743,-0.732342,0.449266,1.326796,-0.047013,0.316033,-0.193839,1.0,0.0,0.0
7996,-2.101475,-0.999569,-2.010781,-1.294788,-0.111930,-1.043489,-0.870653,0.0,0.0,1.0
7997,-1.226871,0.117056,-0.434440,-1.056032,-0.648371,-0.534392,0.043961,0.0,0.0,1.0
7998,1.135377,0.641965,1.213553,-0.397334,1.689627,0.135727,0.823669,1.0,0.0,0.0


In [ ]:
#PCA  
from sklearn.decomposition import PCA

pca = PCA(n_components =0.99)
pca.fit(df1)
X_tr = pca.transform(df1)
X_te=pca.transform(dft)
X_tr.shape

(8000, 9)

In [ ]:
X_tr=pd.DataFrame(X_tr)
X_te=pd.DataFrame(X_te)

In [ ]:
profile = ProfileReport(
    X_tr, title="Profiling Real Adult Data2", html={"style": {"full_width": True}}
)
profile.to_file("real_data_report_ctgan.html")
profile.to_notebook_iframe()

Output hidden; open in https://colab.research.google.com to view.

In [ ]:
# import seaborn as sb
# import matplotlib.pyplot as mp

# dataplot = sb.heatmap(X_tr.corr(), cmap="YlGnBu", annot=True)
  
# # displaying heatmap
# mp.show()

In [ ]:
# y

In [ ]:
scaler = StandardScaler()
y_train=pd.DataFrame(y_train)
y_test=pd.DataFrame(y_test)

# y_train=scaler.fit_transform(y_train)
# y_test=scaler.transform(y_test)

In [ ]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

from sklearn import linear_model


poly = PolynomialFeatures(degree=2)
X_poly_tr = poly.fit_transform(X_tr)
X_poly_te=poly.transform(X_te)

X_poly_te.shape
regression = linear_model.LinearRegression()

model = regression.fit(X_poly_tr, y_train)
score = model.score(X_poly_te, y_test)

In [ ]:
print(score)

0.19387932615142134
